In [205]:
SEMINAR_PATH = '/home/jim/PycharmProjects'

import sys
sys.path.append(SEMINAR_PATH)

import seminar
from seminar.learn.cnn_lstm import convlstm, convgru
from seminar.learn.cnn import LSTM, LSTM2, CNN
from seminar.utils import BiDict

import networkx as nx

import os
import numpy as np
import matplotlib.pyplot as plt
from functools import partial

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

#import dill as pickle
#import pathos.multiprocessing as multiprocessing

In [204]:
from importlib import reload
reload(seminar.learn.cnn)

<module 'seminar.learn.cnn' from '/home/jim/PycharmProjects/seminar/learn/cnn.py'>

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Build Network

In [121]:
trn = '/home/jim/mount/shared/course/geo441/stud/B_Camargue/assembled_datasets/patches/spv__15x15__s1_s2_s2inds_visnirtex_unnorm/trn'
val = '/home/jim/mount/shared/course/geo441/stud/B_Camargue/assembled_datasets/patches/spv__15x15__s1_s2_s2inds_visnirtex_unnorm/val'

#trn = '/home/jim/PycharmProjects/seminar/notebooks/data/spv__17x17__f32__nofilt__nearest_time_upsampled/trn'
#val = '/home/jim/PycharmProjects/seminar/notebooks/data/spv__17x17__f32__nofilt__nearest_time_upsampled/val'

In [122]:
classes = np.unique(list(map(int, [x[1] for x in os.walk(trn)][0] + [x[1] for x in os.walk(val)][0])))

In [123]:
classes

array([1110, 1121, 1122, 1124, 1131, 1132, 1133, 1210, 1230, 1310, 1320,
       1330, 1411, 1412, 1413, 1415, 1420, 2111, 2112, 2113, 2114, 2115,
       2120, 2131, 2132, 2140, 2210, 2220, 2310, 3110, 3121, 3122, 3123,
       3211, 3212, 3220, 3310, 3321, 3322, 3323, 4111, 4112, 4113, 4121,
       4122, 4131, 4132, 4133, 4211, 4212, 4220, 5120, 5210])

## Define Reader and Test It

In [7]:
def reader(path, with_cloud_mask=False):
    patch = np.load(path)

    # s1 bands
    rang = np.array([-20, 5])
    patch[:, :2, :, :] = np.clip((patch[:, :2, :, :] - rang.mean()) / (rang[1] - rang[0]), -1, 1)
    
    # opt bands
    rang = np.array([0, 10000])
    patch[:, 2:12, :, :] = np.clip((patch[:, 2:12, :, :] - rang.mean()) / (rang[1] - rang[0]), -1, 1)
    
    # index evi
    rang = np.array([-20000, 20000])
    patch[:, 12, :, :] = (patch[:, 13, :, :] - rang.mean()) / (rang[1] - rang[0])
    
    nan_val = (-3e4  - rang.mean()) / (rang[1] - rang[0])
    valid_inds = np.where(patch[:, 12, :, :] > nan_val)
    patch[:, 12, :, :][valid_inds] = np.clip(patch[:, 12, :, :][valid_inds], -1, 1)
    
    # other index bands
    rang = np.array([-10000, 10000])
    patch[:, 13:18, :, :] = (patch[:, 13:18, :, :] - rang.mean()) / (rang[1] - rang[0])
    
    nan_val = (-3e4  - rang.mean()) / (rang[1] - rang[0])
    valid_inds = np.where(patch[:, 13:18, :, :] > nan_val)
    patch[:, 13:18, :, :][valid_inds] = np.clip(patch[:, 13:18, :, :][valid_inds], -1, 1)
    
    # tex bands
    rang = np.array([-100, 100])
    patch[:, 18:68, :, :] = np.clip((patch[:, 18:68, :, :] - rang.mean()) / (rang[1] - rang[0]), -1, 1)
    
    # add a cloud layer
    if with_cloud_mask:
        # use some index but not EVI
        clouds = (patch[:, 13, :, :] < nan_val).astype(np.int)[:, None, :, :]
        patch = np.concatenate([patch, clouds], axis=1)
    
    patch = np.concatenate([patch[:, :2], patch[:, 12:]], axis=1)
    return patch

In [187]:
sample_image_path = '/home/jim/mount/shared/course/geo441/stud/B_Camargue/assembled_datasets/patches/spv__15x15__s1_s2_s2inds_visnirtex_unnorm/trn/2115/1010_1397_1025_1412.npy'#
#sample_image_path = '/home/jim/PycharmProjects/seminar/notebooks/data/spv__17x17__f32__nofilt__nearest_time_upsampled/trn/1121/370_1646_383_1659.npy'
sample_image = np.load(sample_image_path)

In [9]:
sample_image.shape

(115, 69, 15, 15)

In [10]:
time = 12

plt.figure()
plt.imshow(reader(sample_image_path2, with_cloud_mask=True)[time,0])
plt.colorbar()
plt.figure()
plt.hist(reader(sample_image_path2, with_cloud_mask=True)[time,0].flatten())

plt.figure()
plt.imshow(reader(sample_image_path2, with_cloud_mask=True)[time,2])
plt.colorbar()
plt.figure()
plt.hist(reader(sample_image_path2, with_cloud_mask=True)[time,2].flatten())

plt.figure()
plt.imshow(reader(sample_image_path2, with_cloud_mask=True)[time,3])
plt.colorbar()
plt.figure()
plt.hist(reader(sample_image_path2, with_cloud_mask=True)[time,3].flatten())

plt.figure()
plt.imshow(reader(sample_image_path2, with_cloud_mask=True)[time,-1])
plt.colorbar()
plt.figure()
plt.hist(reader(sample_image_path2, with_cloud_mask=True)[time,-1].flatten())

NameError: name 'sample_image_path2' is not defined

<Figure size 432x288 with 0 Axes>

## Define Network

In [209]:
run_name = 'lstm__lay_10_10_dset_all__weigths_eq__reduce_channels8'
default_save_path = '/home/jim/PycharmProjects/seminar/notebooks/lightning_logs/' + run_name + '/'

In [210]:
# default used by the Trainer
checkpoint_callback = ModelCheckpoint(
    filepath= default_save_path + 'checkpoints/{epoch}-{val_loss:.2f}',
    save_top_k=5,
    verbose=True,
    monitor='f1/d1/4',
    mode='max',
)

In [211]:
weights1 = np.array([[1, 0.3, 0.2, 0.1, 0.01],
                    [1, 0.3, 0.2, 0.1, 0.01],
                    [1, 0.3, 0.2, 0.1, 0.01],
                    [1, 0.3, 0.2, 0.1, 0.01],
                    [1, 0.3, 0.2, 0.1, 0.01]])

In [212]:
weights2 = np.array([[1, 1, 1, 1, 1]]).transpose()

In [213]:
weights22 = np.array([[1, 1, 1, 1, 1],
                     [1, 1, 1, 1, 1],
                     [1, 1, 1, 1, 1],
                     [1, 1, 1, 1, 1],
                     [1, 1, 1, 1, 1]])

In [214]:
weights22

array([[1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1]])

In [216]:
lstm = LSTM(in_channels=sample_image.shape[1] + 1 - 10,
            num_layers=3, hidden_channels=[5, 5, 1], 
            kernel_size=[(3, 3), (3, 3), (3, 3)],#, (3, 3), (3, 3)],
            train_image_folder=trn,
            val_image_folder=val,
            hierarchy_weights=weights22, seq_len=sample_image.shape[0],
            reduce_hidden=1, reduce_channels=16,
            n_jobs=6, batch_size=32, input_shape=sample_image[0, 0].shape, epoch_size=None, stratified_input=True, reader=partial(reader, with_cloud_mask=True))

In [167]:
lstm2 = LSTM2(input_shape=sample_image.shape[1:], hidden_size=100, embed_size=100, in_channels=sample_image.shape[1] + 1 - 10,
              train_image_folder=trn, val_image_folder=val, n_jobs=3, drop_rate=0.3, bn_momentum=0.9, bn_track_running_stats=False,
              channels=[30, 30], batch_size=32, seq_len=sample_image.shape[0], epoch_size=None, lr=1e-4, 
              hierarchy_weights=weights1, kernel_size=3, reduce_kernel_size=11, reduce_hidden=1, reduce_channels=8, 
              stratified_input=True, reader=partial(reader, with_cloud_mask=True))

In [138]:
cnn = CNN(input_shape=sample_image.shape[1:], linspace=100, in_channels=sample_image.shape[1] + 1 - 10,
              train_image_folder=trn, val_image_folder=val, n_jobs=3, drop_rate=0.3, bn_momentum=0.9, bn_track_running_stats=False,
              channels=[30, 30], batch_size=16, seq_len=sample_image.shape[0], epoch_size=None, lr=1e-4, 
              hierarchy_weights=weights1, kernel_size=3, stratified_input=True, reader=partial(reader, with_cloud_mask=True), reg_invariance=0)

## Run Network

In [ ]:
trainer = Trainer(check_val_every_n_epoch=1, reload_dataloaders_every_epoch=False, 
                  checkpoint_callback=checkpoint_callback, default_save_path=default_save_path, nb_gpu_nodes=12,)
                  #resume_from_checkpoint=os.path.join(default_save_path, 'checkpoints/epoch=24-val_loss=2.42.ckpt'))
                 #train_percent_check=0.25)#,  val_check_interval=100, test_percent_check=0.1, check_val_every_n_epoch=0.25,)
trainer.fit(lstm)

## Test Network